In [1]:
load("cpcode.sage")

In [2]:
q = 7
for k in range(q - 1):
    code = CharacterPolynomialCode(q - 1, k + 1)
    print(code, code.covering_radius())

(6, 1, 6) CP code with character chi_1 associated with [6, 1, 6] Generalized Reed-Solomon Code over GF(7) 5
(6, 2, 5) CP code with character chi_1 associated with [6, 2, 5] Generalized Reed-Solomon Code over GF(7) 4
(6, 3, 4) CP code with character chi_1 associated with [6, 3, 4] Generalized Reed-Solomon Code over GF(7) 3
(6, 4, 3) CP code with character chi_1 associated with [6, 4, 3] Generalized Reed-Solomon Code over GF(7) 2
(6, 5, 2) CP code with character chi_1 associated with [6, 5, 2] Generalized Reed-Solomon Code over GF(7) 1
(6, 6, 1) CP code with character chi_1 associated with [6, 6, 1] Generalized Reed-Solomon Code over GF(7) 0


In [3]:
q = 9
for k in range(1, q - 1):
    code = GRSBase(q - 1, k + 1)
    print(code, code.covering_radius())

(8, 1, 8) CP code with character chi_1 associated with [8, 2, 7] Generalized Reed-Solomon Code over GF(9) 7
(8, 2, 7) CP code with character chi_1 associated with [8, 3, 6] Generalized Reed-Solomon Code over GF(9) 6
(8, 2, 7) CP code with character chi_1 associated with [8, 4, 5] Generalized Reed-Solomon Code over GF(9) 6
(8, 3, 5) CP code with character chi_1 associated with [8, 5, 4] Generalized Reed-Solomon Code over GF(9) 5
(8, 4, 4) CP code with character chi_1 associated with [8, 6, 3] Generalized Reed-Solomon Code over GF(9) 4
(8, 4, 4) CP code with character chi_1 associated with [8, 7, 2] Generalized Reed-Solomon Code over GF(9) 4
(8, 5, 4) CP code with character chi_1 associated with [8, 8, 1] Generalized Reed-Solomon Code over GF(9) 3


In [4]:
for k, d in enumerate(range(8, 1, -1)):
    print(d, d + (k+1)//3, d - 1 + (k+1)//3)

8 8 7
7 7 6
6 7 6
5 6 5
4 5 4
3 5 4
2 4 3


In [5]:
q = 16
for k in range(1, q - 1):
    code = GRSBase(q - 1, k + 1)
    print(code)

(15, 1, 15) CP code with character chi_1 associated with [15, 2, 14] Generalized Reed-Solomon Code over GF(16)
(15, 1, 15) CP code with character chi_1 associated with [15, 3, 13] Generalized Reed-Solomon Code over GF(16)
(15, 2, 14) CP code with character chi_1 associated with [15, 4, 12] Generalized Reed-Solomon Code over GF(16)
(15, 2, 14) CP code with character chi_1 associated with [15, 5, 11] Generalized Reed-Solomon Code over GF(16)
(15, 3, 13) CP code with character chi_1 associated with [15, 6, 10] Generalized Reed-Solomon Code over GF(16)
(15, 3, 13) CP code with character chi_1 associated with [15, 7, 9] Generalized Reed-Solomon Code over GF(16)
(15, 4, 12) CP code with character chi_1 associated with [15, 8, 8] Generalized Reed-Solomon Code over GF(16)
(15, 4, 12) CP code with character chi_1 associated with [15, 9, 7] Generalized Reed-Solomon Code over GF(16)
(15, 5, 11) CP code with character chi_1 associated with [15, 10, 6] Generalized Reed-Solomon Code over GF(16)
(15,

In [6]:
# test decoder
q = 17
k = q//2 + 1
cpcode = CharacterPolynomialCode(q - 1, k + 1)
x = cpcode.polynomial_ring().gen()
f = x^k + (q-1)*x  # this gives the minimum weight grs codeword
f in cpcode

True

In [7]:
cw = cpcode.encode(f)
cw

(1.00000000, -0.602634636 - 0.798017228*I, 1.00000000, 0.445738356 - 0.895163291*I, 1.00000000, -0.850217136 + 0.526432163*I, 1.00000000, -0.273662990 - 0.961825643*I, 1.00000000, -0.602634636 + 0.798017227*I, 1.00000000, 0.445738356 + 0.895163291*I, 1.00000000, -0.850217135 - 0.526432163*I, 1.00000000, -0.273662990 + 0.961825643*I)

In [8]:
cw in cpcode

True

In [9]:
cpcode.decode(cw) == f

True

In [10]:
cpcode.list_decode(cw)

[x^9 + 16*x]

In [11]:
cp_channel = channels.StaticErrorRateChannel(
    CC^cpcode.length, 
    0
)
rw = cp_channel.transmit_unsafe(cw)
rw in cpcode

True

In [12]:
cp_channel = channels.StaticErrorRateChannel(
    CC^cpcode.length, 
    1
)
rw = cp_channel.transmit_unsafe(cw)
rw in cpcode

False

In [13]:
cp_channel = channels.StaticErrorRateChannel(
    CC^cpcode.length, 
    cpcode.list_decoding_radius()
)
rw = cp_channel.transmit_unsafe(cw)
rw in cpcode

False

In [14]:
%%time
# test list decoder
import numpy as np


NUM_POLS = 5
NUM_ERRS = 200

for q in Primes():
    if q > 5:
        if q < 30:
            k = q//2 + 1
            cpcode = CharacterPolynomialCode(q - 1, k + 1)
            cp_lsizes = []
            rs_lsizes = []
            num_errors = cpcode.list_decoding_radius()
            for _ in range(NUM_POLS):
                f = cpcode.polynomial_ring().random_element(degree=k)
                g = cpcode.convert_polynomial(f)
                cp_cw = cpcode.encode(g)  # cp codeword
                rs_cw = cpcode.grs.encoder("EvaluationPolynomial").encode(g)  # rs codeword
                cp_channel = channels.StaticErrorRateChannel(
                    CC^cpcode.length, 
                    num_errors
                )
                rs_channel = channels.StaticErrorRateChannel(
                    cpcode.field^cpcode.length, 
                    num_errors
                )
                gsd = cpcode.grs.decoder("GuruswamiSudan", tau=num_errors)
                for _ in range(NUM_ERRS):
                    cp_rw = cp_channel.transmit_unsafe(cp_cw)
                    rs_rw = rs_channel.transmit_unsafe(rs_cw)
                    cp_ls = cpcode.list_decode(cp_rw)
                    rs_ls = gsd.decode_to_message(rs_rw)
                    if len(cp_ls) > 1:
                        print(cpcode, f, g, cp_cw, rs_cw, 
                              cp_rw, rs_rw, cp_ls, rs_ls, 
                              sep="\n\t")
                    cp_lsizes.append(len(cp_ls))
                    rs_lsizes.append(len(rs_ls))
            print(q, np.mean(cp_lsizes), np.mean(rs_lsizes))
        else:
            break

7 1.0 6.0
11 1.0 3.808
13 1.0 1.0
17 1.0 1.0
19 1.0 1.178
23 1.0 1.025
29 1.0 1.031
CPU times: user 4min 1s, sys: 336 ms, total: 4min 1s
Wall time: 4min 2s
